In [3]:
#Copyright (c) Microsoft. All rights reserved.
import logging
import os
import azure.cognitiveservices.speech as speechsdk
from pydub import AudioSegment
from langchain.docstore.document import Document
from scipy.io import wavfile
import time

import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from openai import AzureOpenAI

c:\Users\asthaagarwal\AppData\Local\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
# Speech to English text for audio file
def translation_once_from_file(filename):
    """performs one-shot speech translation from input from an audio file"""
    # <TranslationOnceWithFile>
    # set up translation parameters: source language and target languages
    translation_config = speechsdk.translation.SpeechTranslationConfig(
        subscription="0ab292a969f64bcd91883645944bb66d", region= "eastus",
        speech_recognition_language='kn-IN')
    translation_config.add_target_language("en")
    translation_config.add_target_language("fr")
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    # Creates a translation recognizer using and audio file as input.
    recognizer = speechsdk.translation.TranslationRecognizer(
        translation_config=translation_config, audio_config=audio_config)

    # Starts translation, and returns after a single utterance is recognized. The end of a
    # single utterance is determined by listening for silence at the end or until a maximum of 15
    # seconds of audio is processed. The task returns the recognition text as result.
    # Note: Since recognize_once() returns only a single utterance, it is suitable only for single
    # shot recognition like command or query.
    # For long-running multi-utterance recognition, use start_continuous_recognition() instead.
    result = recognizer.recognize_once()

    # Check the result
    if result.reason == speechsdk.ResultReason.TranslatedSpeech:
        print("TranslatedSpeech Recognized")
        text = result.translations['en']
    elif result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        print("Translation canceled: {}".format(result.cancellation_details.reason))
        if result.cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(result.cancellation_details.error_details))

    text = result.translations['en']
    # docs = [Document(page_content=text)]
    return text

In [5]:
service_endpoint =  
index_name =    #os.getenv("AZURE_SEARCH_INDEX_NAME")
key =   
open_ai_endpoint =
open_ai_key =  
openai_api_version =   '2024-05-01-preview'
embed_model_id =  "text-embedding-ada-002"

In [6]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)



ImportError: cannot import name 'AzureOpenAIParameters' from 'azure.search.documents.indexes.models' (c:\Users\asthaagarwal\AppData\Local\anaconda3\Lib\site-packages\azure\search\documents\indexes\models\__init__.py)

In [2]:

# Create a search index
index_client = SearchIndexClient(
    endpoint=open_ai_endpoint, credential=open_ai_key)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1532, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1532, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myVectorizer",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=open_ai_endpoint,
                deployment_id=embed_model_id,
                model_name=embed_model_id,
                api_key=open_ai_key
            )
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

NameError: name 'open_ai_endpoint' is not defined

In [25]:
# Load data in cognitive search

# Create index in cognitive search
def create_index(name: str):
    from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchAlgorithmConfiguration,
    )

    fields = [
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="description", type=SearchFieldDataType.String),
        SearchField(
            name="DescriptionVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            dimensions=1536,
            vector_search_configuration="default",
        )
    ]
    vector_search = VectorSearch(
        algorithm_configurations=[VectorSearchAlgorithmConfiguration(name="default", kind="hnsw")]
    )
    return SearchIndex(name=name, fields=fields, vector_search=vector_search)

# Prepare documents for ingestion in cognitive search index
def get_documents(filenames):
    docs=[]
    for filename in filenames:
        # Extract data from audio file
        text = translation_once_from_file(filename)
        docs.append(
            {
                "Id": filename,
                "description": text,
                "DescriptionVector": get_embeddings(text)
            }
        )
    return docs

In [26]:
index = create_index(index_name)

ImportError: cannot import name 'VectorSearchAlgorithmConfiguration' from 'azure.search.documents.indexes.models' (c:\Users\asthaagarwal\AppData\Local\anaconda3\Lib\site-packages\azure\search\documents\indexes\models\__init__.py)

In [ ]:
def getSearchClient(index_name):
    credential = AzureKeyCredential(key)
    # index_client = SearchIndexClient(service_endpoint, credential)
    # print(index_name)

    # index = create_index(index_name)


    # print(index)
    # index_client.create_index(index)

    client = SearchClient(service_endpoint, index_name, credential)
    # print(client)
    return client